In [1]:
from inspect import Parameter
from turtle import color
from unittest import result
from isort import code, file
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd
from datetime import time
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from itertools import *
import collections

# Read Excel Files
all_jobs = pd.read_excel("LastQueue.xlsx", sheet_name="AllJobs")
all_queue_items = pd.read_excel("LastQueue.xlsx", sheet_name="Sayfa1")

# create a list containing the name of unique tasks
items = all_queue_items.ExecutorJobId.unique().tolist()
items.remove(11018.0)
items_list = sorted(items)
# create a dataframe using the file QueueItems_Full and make necessary adjustments
all_queue_df = all_queue_items[
    all_queue_items["ExecutorJobId"].isin(items_list)
].sort_values(by="ExecutorJobId", ascending=True)
all_queue_df = all_queue_df.dropna(subset=["ExecutorJobId"], how="all")
# create a dataframe using the file All_Jobs1 and make necessary adjustments
all_jobs_df = all_jobs[all_jobs["Id"].isin(items_list)].sort_values(
    by="Id", ascending=True
)
all_jobs_df = all_jobs_df[:-1]

# create periods
starting = time(9, 0, 0)
mid = time(12, 30, 0)
afternoon = time(15, 30, 0)
ending = time(18, 30, 0)

# converting time columns to datetime
all_queue_df["StartProcessing"] = pd.to_datetime(
    all_queue_df["StartProcessing"], format="%d/%m/%Y"
)
all_queue_df["EndProcessing"] = pd.to_datetime(
    all_queue_df["EndProcessing"], format="%d/%m/%Y"
)
all_jobs_df["CreationTime"] = pd.to_datetime(
    all_jobs_df["CreationTime"], format="%d/%m/%Y"
)

all_jobs_df["creation_time"] = [d.time() for d in all_jobs_df["CreationTime"]]
all_queue_df["new_date_start"] = [d.date() for d in all_queue_df["StartProcessing"]]
all_queue_df["new_time_start"] = [d.time() for d in all_queue_df["StartProcessing"]]
all_queue_df["new_date_end"] = [d.date() for d in all_queue_df["EndProcessing"]]
all_queue_df["new_time_end"] = [d.time() for d in all_queue_df["EndProcessing"]]

period_list = ['WH','NH']

In [2]:
# add the robot names to the dataframe
robo_dic = {}
for i in range(len(all_jobs_df)):
    j = all_jobs_df.iloc[i]["Id"]
    r = all_jobs_df.iloc[i]["RobotName"]
    robo_dic[j] = r

In [3]:
robo_name_list= []
robo_id_name_dict = {}
del_list = []
for i in range(len(all_queue_df)):
    jobs = all_queue_df.iloc[i]["ExecutorJobId"]
    if jobs in robo_dic.keys():
        robo_name_list.append(robo_dic[jobs])
    else:
        del_list.append(i) 
        
            


In [4]:
print(len(robo_name_list), len(all_queue_df),len(del_list))

71822 71855 33


In [5]:
new_que_df = all_queue_df.copy()
for i in range(len(del_list)):
    new_que_df = new_que_df.drop(new_que_df.index[del_list[i]-i])

In [6]:
new_que_df["RobotName"] = robo_name_list

In [7]:
print(new_que_df)

       ExecutorJobId  QueueDefinitionId  Status   StatusName  RobotId  \
54094            747                 34       2       Failed       15   
37334            747                 34       3   Successful       15   
14124            747                 34       3   Successful       15   
25520            747                 34       3   Successful       15   
35222            747                 34       3   Successful       15   
...              ...                ...     ...          ...      ...   
18337          12943                 34       3   Successful       15   
843            12943                 34       3   Successful       15   
17360          12943                 34       3   Successful       15   
28282          12943                 34       3   Successful       15   
20273          12943                 34       3   Successful       15   

              StartProcessing           EndProcessing  \
54094 2022-03-18 14:35:27.420 2022-03-18 14:35:33.183   
37334 202

In [8]:
processes = new_que_df.ProcessID.unique().tolist()
robots = new_que_df.RobotName.unique().tolist()

In [9]:
jobid = new_que_df.ExecutorJobId.unique().tolist()
print(len(processes))

32


In [10]:
##create a model
model = pyo.ConcreteModel()
model.transactions = pyo.Set(initialize=processes) #P
model.periods = pyo.Set(initialize=period_list) #K
model.robots = pyo.Set(initialize=robots) #R

In [11]:
task_time ={}
# Create execution time for each job
for i in range(len(new_que_df)):
    if new_que_df.iloc[i]["StatusName"] != "Failed":
        process_id = new_que_df.iloc[i]["ProcessID"]
        if process_id in task_time.keys():
            task_time[process_id] += (
            new_que_df.iloc[i]["EndProcessing"] - new_que_df.iloc[i]["StartProcessing"]
            ).total_seconds()
        else:
            task_time[process_id] = (
            new_que_df.iloc[i]["EndProcessing"] - new_que_df.iloc[i]["StartProcessing"]
            ).total_seconds()
model.tp = pyo.Param(model.transactions,initialize=task_time)
Tp = model.tp

In [12]:
volume = new_que_df.ProcessID.value_counts().to_dict()
model.Vp = pyo.Param(model.transactions, initialize=volume)
Vp = model.Vp

In [13]:
# Wkp 1 executed, 0 not executed
midnight = (0,0,0)
wkp = {}
for i in range(len(new_que_df)):
    wkp[period_list[0],new_que_df.iloc[i]["ProcessID"]] = 0
    wkp[period_list[1],new_que_df.iloc[i]["ProcessID"]] = 0
        
for i in range(len(new_que_df)):
    begin = new_que_df.iloc[i]["new_time_start"]
    end = new_que_df.iloc[i]["new_time_end"]
    if  (begin >= starting and begin <= ending) or (end >= starting and end <= ending):
        wkp[period_list[0],new_que_df.iloc[i]["ProcessID"]] = 1

    if (begin < starting or begin > ending) or ((end < starting or end > ending)):
        wkp[period_list[1],new_que_df.iloc[i]["ProcessID"]] = 1

In [14]:
model.Wkp = pyo.Param(model.periods,model.transactions,initialize=wkp)
Wkp = model.Wkp

In [15]:
#N: minimal number of robots required
model.N = pyo.Var(initialize= len(robots))
N = model.N

In [16]:
# Lk length of period k
working_length = 34200
model.Lk = pyo.Param(model.periods,initialize={period_list[0]:34200,period_list[1]:(86400-34200)})
Lk = model.Lk

In [17]:
# Xrkp: number of transactions of type p ∈ P processed by robot r at period k ∈ K.
model.X_rkp = pyo.Var(model.robots, model.periods, model.transactions, within= pyo.NonNegativeIntegers)
X_rkp = model.X_rkp

In [19]:
#create a variable called Nk, which is the number of robots required at period k ∈ K
model.Nk = pyo.Var(model.periods,domain= pyo.NonNegativeIntegers,initialize=5)
Nk = model.Nk

In [20]:
def Obj_Func(model):
    return sum(Nk[k] for k in model.periods)

model.obj = pyo.Objective(rule=Obj_Func, sense= pyo.minimize)

def Constraint1(model, transactions):
    return sum(sum(model.X_rkp[r,k,transactions] * model.Wkp[k, transactions]for k in model.periods) for r in model.robots) == model.Vp[transactions]
model.Const1 = pyo.Constraint(model.transactions,rule=Constraint1)

def Constraint2(model, periods):
    return sum(sum(model.X_rkp[r,periods,p] * model.tp[p] for p in model.transactions) for r in model.robots) <= model.Lk[periods]* model.Nk[periods]
model.Const2 = pyo.Constraint(model.periods,rule=Constraint2)

def Constraint3(model,periods):
    return (model.N - model.Nk[periods]) >= 0
model.Const3 = pyo.Constraint(model.periods, rule=Constraint3)

def Constraint4(model, periods): 
    return sum(sum(model.X_rkp[r,periods,p] for p in model.transactions)for r in model.robots) >= model.Nk[periods]
model.Const4 = pyo.Constraint(model.periods, rule=Constraint4)
#get the result
Solver = SolverFactory('glpk')
results = Solver.solve(model)
print(results)
print("Objective Function: ", model.obj())

Objective Function:  10
